<a href="https://colab.research.google.com/github/sbakhit/Modified-MNIST/blob/master/CNN_94.5_Anton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch.nn as nn


class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        # Defining a 2D convolution layer
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer8 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer9 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer11 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.drop_out = nn.Dropout()

        # Defining another 2D convolution layer
        self.fc1 = nn.Sequential(
            nn.Linear(1*1*256, 4096)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 512)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [0]:
#!/usr/bin/env python3

# FOR TESTING ON VALIDATION DATASET

import numpy as np
import torch
import pandas as pd
import cv2
from torch import nn, optim
from time import time
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from matplotlib import pyplot as plt

# number of times we iterate over the training set
EPOCHS = 40
LEARNING_RATE = 0.001
BATCH_SIZE = 512
TRAIN_PATH = '/content/drive/My Drive/McGill/comp551/data/train_max_x'
TEST_PATH = '/content/drive/My Drive/McGill/comp551/data/test_max_x'
TARGETS_PATH = '/content/drive/My Drive/McGill/comp551/data/train_max_y.csv'
THRESH = 240

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def main():
    print('torch.cuda.device_count(): {0}'.format(torch.cuda.device_count()))
    print('torch.cuda.get_device_name: {0}'.format(torch.cuda.get_device_name(0)))
    # load images as a numpy array
    train_dataset = np.array(np.load(TRAIN_PATH, allow_pickle=True))
    train_dataset = np.array([cv2.threshold(i, THRESH, 255, cv2.THRESH_BINARY)[1] for i in train_dataset])
    train_dataset = train_dataset / 255.0

    targets = pd.read_csv(TARGETS_PATH, delimiter=',', skipinitialspace=True)
    targets = targets.to_numpy()
    # remove id column
    targets = targets[:, 1]
    targets = targets.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(train_dataset, targets, test_size=0.2, random_state=42)
    # Clean memory
    len_train_dataset = len(X_train)
    train_dataset = None
    targets = None

    X_train = torch.from_numpy(X_train)
    y_train = torch.from_numpy(y_train)
    X_test = torch.from_numpy(X_test)
    y_test = torch.from_numpy(y_test)
    y_train = y_train.long()
    y_test = y_test.long()

    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])

    train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=8)
    test_dataset = torch.utils.data.TensorDataset(X_test,y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=8)

    # defining the model
    model = Net().to(device)

    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    time0 = time()

    for e in range(EPOCHS):
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # clearing the Gradients of the model parameters
        optimizer.zero_grad()

        # prediction for training
        output_train = model(inputs)
        
        # computing the training loss
        loss_train = criterion(output_train, labels)

        # computing the updated weights of all the model parameters
        loss_train.backward()

        # And optimizes its weights here
        optimizer.step()

        running_loss += loss_train.item()
      else:
            print("Epoch {} - Training loss: {}".format(e, running_loss / len_train_dataset))

    print("\nTraining Time (in minutes) =", (time() - time0) / 60)

    # prediction for validation set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # accuracy on validation set
    print("\nModel Accuracy =", (100 * correct / total))


if __name__ == '__main__':
    main()

In [3]:
#!/usr/bin/env python3

# FOR MAKING PREDICTIONS ON TEST SET AND SAVING TO CSV

import numpy as np
import torch
import pandas as pd
import cv2
import shutil
from torch import nn, optim
from time import time
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from matplotlib import pyplot as plt

# number of times we iterate over the training set
EPOCHS = 100
LEARNING_RATE = 0.001
BATCH_SIZE = 256
TRAIN_PATH = '/content/drive/My Drive/McGill/comp551/data/train_max_x'
TEST_PATH = '/content/drive/My Drive/McGill/comp551/data/test_max_x'
TARGETS_PATH = '/content/drive/My Drive/McGill/comp551/data/train_max_y.csv'
THRESH = 240

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def main():
    print('torch.cuda.device_count(): {0}'.format(torch.cuda.device_count()))
    print('torch.cuda.get_device_name: {0}'.format(torch.cuda.get_device_name(0)))
    # load images as a numpy array
    X_train = np.array(np.load(TRAIN_PATH, allow_pickle=True))
    X_train = np.array([cv2.threshold(i, THRESH, 255, cv2.THRESH_BINARY)[1] for i in X_train])
    X_train = X_train / 255.0

    X_test = np.array(np.load(TEST_PATH, allow_pickle=True))
    X_test = np.array([cv2.threshold(i, THRESH, 255, cv2.THRESH_BINARY)[1] for i in X_test])
    X_test = X_test / 255.0

    y_train = pd.read_csv(TARGETS_PATH, delimiter=',', skipinitialspace=True)
    y_train = y_train.to_numpy()
    # remove id column
    y_train = y_train[:, 1]
    y_train = y_train.astype(int)

    # Clean memory
    len_train_dataset = len(X_train)

    X_train = torch.from_numpy(X_train)
    y_train = torch.from_numpy(y_train)
    X_test = torch.from_numpy(X_test)
    y_train = y_train.long()

    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])

    train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=8)
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=BATCH_SIZE, num_workers=8)

    # defining the model
    model = Net().to(device)

    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    time0 = time()

    for e in range(EPOCHS):
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # clearing the Gradients of the model parameters
        optimizer.zero_grad()

        # prediction for training
        output_train = model(inputs)
        
        # computing the training loss
        loss_train = criterion(output_train, labels)

        # computing the updated weights of all the model parameters
        loss_train.backward()

        # And optimizes its weights here
        optimizer.step()

        running_loss += loss_train.item()
      else:
            print("Epoch {} - Training loss: {}".format(e, running_loss / len_train_dataset))

    print("\nTraining Time (in minutes) =", (time() - time0) / 60)

    # prediction for test set
    predictions = np.array([])
    with torch.no_grad():
      for images in test_loader:
        images = images.to(device)
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        predictions = np.append(predictions, predicted.cpu().numpy())

    predictions = predictions.astype(int)
    Id = list(range(0, 10000))
    Id = np.asarray(Id)
    results = pd.DataFrame({'Id': Id, 'Label': predictions}, columns=['Id', 'Label'])
    from google.colab import files
    results.to_csv('predictions.csv', index=False)
    shutil.move("/content/predictions.csv", "/content/drive/My Drive/McGill/comp551/predictions.csv")
    print('Results have been saved to predictions.csv')

if __name__ == '__main__':
    main()

torch.cuda.device_count(): 1
torch.cuda.get_device_name: Tesla K80
Epoch 0 - Training loss: 0.008411275391578675
Epoch 1 - Training loss: 0.006557975809574127
Epoch 2 - Training loss: 0.005190859088897705
Epoch 3 - Training loss: 0.003582865102291107
Epoch 4 - Training loss: 0.002622839918732643
Epoch 5 - Training loss: 0.0020789412784576414
Epoch 6 - Training loss: 0.0017250763577222824
Epoch 7 - Training loss: 0.0014393350622057914
Epoch 8 - Training loss: 0.0012748295485973359
Epoch 9 - Training loss: 0.0011436630353331566
Epoch 10 - Training loss: 0.0010539979302883148
Epoch 11 - Training loss: 0.0009986692982912064
Epoch 12 - Training loss: 0.0008758644995093346
Epoch 13 - Training loss: 0.0008017231269180775
Epoch 14 - Training loss: 0.0007682373896986246
Epoch 15 - Training loss: 0.0007050850105285645
Epoch 16 - Training loss: 0.0006293134747445584
Epoch 17 - Training loss: 0.0006058661715686322
Epoch 18 - Training loss: 0.0005483870671689511
Epoch 19 - Training loss: 0.00052082